In [19]:
import os

subj = "Subj3"
exp = "reproduction"
os.makedirs(f"{subj}/{exp}", exist_ok = True)

### Imports

In [20]:
%load_ext autoreload
%autoreload 2

import pandas
import sklearn.preprocessing
import sklearn.decomposition

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Features

In [21]:
df_ft_psd_loc_db = pandas.read_feather(f'{subj}/src/df_ft_psd_loc_db.feather')
df_ft_psd_ind_loc_log = pandas.read_feather(f'{subj}/src/df_ft_psd_ind_loc_log.feather')
df_ft_coh_ind_loc = pandas.read_feather(f'{subj}/src/df_ft_coh_ind_loc.feather')
df_ft_plv_ind_loc = pandas.read_feather(f'{subj}/src/df_ft_plv_ind_loc.feather')

In [22]:
# PSD & PSD indices
ft_psd_loc_db_sc = sklearn.preprocessing.StandardScaler().fit_transform(df_ft_psd_loc_db.to_numpy())
df_ft_psd_loc_db_sc = pandas.DataFrame(ft_psd_loc_db_sc, columns = df_ft_psd_loc_db.columns)

ft_psd_ind_loc_sc = sklearn.preprocessing.StandardScaler().fit_transform(df_ft_psd_ind_loc_log.to_numpy())
df_ft_psd_ind_loc_sc = pandas.DataFrame(ft_psd_ind_loc_sc, columns = df_ft_psd_ind_loc_log.columns)

# Coherence indices
ft_coh_ind_loc_sc = sklearn.preprocessing.StandardScaler().fit_transform(df_ft_coh_ind_loc.to_numpy())
df_ft_coh_ind_loc_sc = pandas.DataFrame(ft_coh_ind_loc_sc, columns = df_ft_coh_ind_loc.columns)

# PLV indices
ft_plv_ind_loc_sc = sklearn.preprocessing.StandardScaler().fit_transform(df_ft_plv_ind_loc.to_numpy())
df_ft_plv_ind_loc_sc = pandas.DataFrame(ft_plv_ind_loc_sc, columns = df_ft_plv_ind_loc.columns)

### PCA

In [23]:
df_ft_tot_orig = pandas.concat([ df_ft_psd_loc_db_sc, df_ft_psd_ind_loc_sc, df_ft_coh_ind_loc_sc, df_ft_plv_ind_loc_sc ], axis = 1)
print(len(df_ft_tot_orig.columns))

N_COMP = 15
pca = sklearn.decomposition.PCA(n_components = N_COMP, random_state = 0)
ft_tot_pca = pca.fit_transform(df_ft_tot_orig)

print(ft_tot_pca.shape)
print('Explained variance', round(pca.explained_variance_ratio_.sum(), 2))
print([ round(x, 3) for x in pca.explained_variance_ratio_ ])

pca_comp_names = [ 'PCA_' + str(i) for i in range(len(ft_tot_pca[0, :])) ]
if subj != 'Subj2': df_ft_tot_pca = pandas.DataFrame(ft_tot_pca, columns = pca_comp_names)
else: df_ft_tot_pca = pandas.DataFrame(ft_tot_pca[:-2], columns = pca_comp_names) # for m8 (artifacts at the end)
print(df_ft_tot_pca.shape)

765
(1180, 15)
Explained variance 0.72
[0.323, 0.095, 0.07, 0.043, 0.032, 0.025, 0.023, 0.02, 0.018, 0.016, 0.014, 0.012, 0.011, 0.01, 0.009]
(1180, 15)


### Save

In [24]:
df_ft_tot_pca.to_feather(f"{subj}/{exp}/features.feather")
display(df_ft_tot_pca)

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12,PCA_13,PCA_14
0,11.728765,-6.525690,5.859528,4.697581,2.683782,-4.976401,-10.792175,6.038854,11.123139,-3.223298,-6.756775,1.539722,4.479549,-0.995040,-0.123784
1,6.533463,-3.984159,5.820724,4.287549,-4.961654,-5.173978,-11.895655,7.375446,10.113042,-3.551145,-2.459928,3.073350,6.564170,-1.332062,1.392719
2,11.577539,-1.883571,3.730306,2.878780,1.171641,-1.659451,-11.137466,5.856929,11.982865,-2.758356,-4.523081,3.038256,4.995357,0.954625,0.316057
3,8.584325,3.816201,3.296246,5.040570,2.967130,-0.964272,-12.376489,0.018549,6.973252,-1.887550,-4.034465,3.479027,8.295133,-1.074692,-0.743376
4,11.424313,8.332378,6.532666,7.850389,4.199225,4.353842,-12.705099,0.033872,8.783815,-4.209234,-4.650551,3.006022,3.737987,-1.904677,-2.333160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,-24.313129,8.676558,-3.420483,-0.444838,0.090556,1.824741,-0.683174,-6.695199,-2.881181,0.076896,1.220176,-0.904387,-2.031414,1.589136,2.703787
1176,-31.827418,-0.173498,-3.489611,4.265456,-2.095619,2.775914,-0.589848,0.695378,-2.286300,0.693892,3.336475,-2.182549,-3.524272,1.490769,1.031317
1177,-24.587065,-5.597988,-8.130704,2.082220,-0.813676,5.051108,-0.814790,-2.946510,-1.022510,2.111610,1.857554,0.899073,-3.660419,0.080238,1.698454
1178,-18.762201,-5.152575,-8.189062,0.628492,-0.848656,4.557355,-0.670813,-3.269185,-0.889240,4.132427,-0.858916,1.486501,-5.182440,-3.035938,-0.039190
